In [ ]:
#@title { form-width: "2%" }
#@markdown 

import os, re, ast, json, codecs, csv, IPython, sys, time, pytz
import ipywidgets as widgets
from bs4 import BeautifulSoup
from datetime import  datetime
from IPython.display import display
import pandas as pd
import panel
import numpy as np
panel.extension('tabulator')

tz = pytz.timezone('Turkey')
os.environ['TZ'] = 'Turkey'
env_colab = 'google.colab' in sys.modules
if env_colab:
    import google.colab as colab
nowstr = lambda separete = False: datetime.now(tz).strftime("%y-%m-%d-%H-%M-%S") if separete else str(round(time.time()))
debugbuttons = False

def append_lines(filename,  line = '', lines = None):
    if lines or line:
        with open(filename, "a+") as file:
            file.seek(0)
            data = file.read(100)
            if len(data) > 0:
                file.write("\n")

            if line:
                file.write(line + "\n")

            if lines:
                for i in range(len(lines)):
                    file.write(lines[i])
                    if i < len(lines) - 1:
                        file.write("\n")
        return True
    return False 
  
log_info = "Info"
log_warning = "Warning"
log_error = "Error"
main = "Main"

if env_colab:
    time.tzset()
    app_base = '/content/td_logs'
    log_file = f"/content/td_logs/td_log_{nowstr(True)}.txt"
    log_drv = '/content/drive/MyDrive/General/td_logs'
else:
    app_base = 'c://td/logs'
    log_file = f'c://td/logs/td_log_{nowstr(True)}.txt'
    log_drv = ''

os.makedirs(app_base, exist_ok = True)

def logmaker():
    tchnew = datetime.now().timestamp()
    tck = 1684336351.868823
    df = 86400
    chk = tchnew-tck <= df
    return True

class Logger:
    _instance = None
    @staticmethod
    def getInstance():
        if Logger._instance == None:
            Logger()
        return Logger._instance
  
    def __init__(self):
        if Logger._instance != None:
            raise Exception("Logger is a singleton.")
        else:
            Logger._instance = self
    
    logs = []
    def init(self, hold = False):
        if hold:
            self.logs.append(self.makelog('Log started.', log_info, main))
        else:
            self.log( 'Log started.', log_info,  main)
  
    def makelog(self, text: str, log_type: str = 'Info', source = main) -> str:
        now = datetime.now(tz).strftime("%y:%m:%d:%H:%M:%S")
        source = source.replace('_', ' ').title()
        start = "{} {:^7} {}:". format(now, log_type, source)
        text = text.replace("\n", f"\n{start} ")
        lg = f"{start} {text}"
        return lg
    
    def queue(self, text: str, log_type: str = 'Info', source = main):
        self.logs.append(self.makelog(text, log_type = log_type, source = source))
  
    def info(self, text: str = "", lines = None):
        src = sys._getframe(1).f_code.co_name
        if src == '<module>':
            src = main

        if lines:
            for line in lines:
                self.queue(line, log_info, src)
        if text:
            self.queue(text, log_info, src)
  
    def warning(self, text: str = "", lines = None):
        src = sys._getframe(1).f_code.co_name
        if src == '<module>':
            src = main

        if lines:
            for line in lines:
                self.queue(line, log_warning, src)
        if text:
            self.queue(text, log_warning, src)  
  
    def error(self, text: str = "", lines = None):
        src = sys._getframe(1).f_code.co_name
        if src == '<module>':
            src = main 

        if lines:
            for line in lines:
                self.queue(line, log_error, src)
        if text:
            self.queue(text, log_error, src)
    
    def release(self):
        if (self.logs):
            if self.log(logs = self.logs):
                self.logs.clear()
            else:
                print("Logger: An error happend while trying to write log. logs are preserved.")
  
    def log(self, text = '', log_type= log_info,  source = main, logs = None):
        if not logs and not text:
            return True
        res = False
        if text:
            text = self.makelog(text, log_type, source)
        res = append_lines(filename = log_file, line = text, lines = logs)

        if res:
            self.savelogtodrive()
            return True
        else:
            print("Logger: log: An error happened while trying to write log.")
            return False
  
    def savelogtodrive(self):
        if log_drv and os.path.exists(log_drv) and os.path.isfile(log_file):
            if env_colab:
                !rsync -I "$log_file" "$log_drv_file"

logger = Logger.getInstance()   
info = logger.info
error = logger.error
warning = logger.warning
release = logger.release

if not os.path.isfile(log_file):
    logger.init(True)
    info(f"New Session.")

skip = lambda c: True if c == "" or c == "s" else False
def ext (c):
    c = c.lower()
    return True if c == " " or c == "q" else False

def no(c):
    c = c.lower()
    return True if c == "n" else False

def getattrs(obj):
    attrs_ = []
    for attr in dir(obj):
        if not attr.startswith("__"):
            attrs_.append((attr, getattr(obj, attr)))
    return attrs_

def dump(obj, padding = True, filter = True):
    pad = "              " if padding else ""
    for attr in dir(obj):
        if filter and attr.startswith("__"):
            continue
        print("%s%s =  %r" % (pad, attr, getattr(obj, attr)))

def getattrstr(obj, filter = True, filter_empty = False, add_line = False):
    attrstr = ''
    pre = '\n' if add_line else ', '
    i = True
    for attr in dir(obj):
        if filter and attr.startswith("__"):
            continue
        val = getattr(obj, attr)
        if filter_empty and not val:
            continue
        if i :
            attrstr += "%s =  %r" % (attr, val)
            i = False
        else:
            attrstr += "%s%s =  %r" % (pre, attr, getattr(obj, attr))
    return attrstr
  

def list_lines(lines, str = False):
    if lines:
        if str:
            rtn = ''
            for r in lines:
                if r:
                    rtn += (r +'\n')
            return rtn
        if not str:  
            for line in lines:
                print(line)


contains = lambda string, search: search.lower() in string.lower()
time_str = lambda sec: time.strftime('%H:%M:%S', time.gmtime(sec))

Nested = 'Nested'
ParentSKU = 'ParentSKU'
SharedSKU = 'SharedSKU'
MapAsSimple = 'MapAsSimple'

UsePropertyList = 'UsePropertyList'
UseAttributeTagProperties = 'UseAttributeTagProperties'

wocommerce_header = [ 'ID', 'Type', 'SKU', 'Name', 'Published', 'Is featured?', 'Visibility in catalog', 'Short description', 'Description', 'Date sale price starts', 'Date sale price ends', 'Tax status', 'Tax class', 'In stock?', 'Stock', 'Backorders allowed?', 'Sold individually?', 'Weight (kg)', 'Length (cm)', 'Width (cm)', 'Height (cm)', 'Allow customer reviews?', 'Purchase note', 'Sale price', 'Regular price', 'Categories', 'Tags', 'Shipping class', 'Images', 'Download limit', 'Download expiry days', 'Parent', 'Grouped products', 'Upsells', 'Cross-sells', 'External URL', 'Button text', 'Position', 'Attribute 1 name', 'Attribute 1 value(s)', 'Attribute 1 visible', 'Attribute 1 global', 'Attribute 2 name', 'Attribute 2 value(s)', 'Attribute 2 visible', 'Attribute 2 global', 'Meta: _wpcom_is_markdown', 'Download 1 name', 'Download 1 URL', 'Download 2 name', 'Download 2 URL']
defaultvalues = {"ID":  "", "Type":  "", "SKU":  "", "Name":  "", "Published":  "1", "Is featured?":  "0", "Visibility in catalog":  "visible", "Short description":  "", "Description":  "", "Date sale price starts":  "", "Date sale price ends":  "", "Tax status":  "taxable", "Tax class":  "", "In stock?":  "1", "Stock":  "", "Backorders allowed?":  "0", "Sold individually?":  "0", "Weight (kg)":  "", "Length (cm)":  "", "Width (cm)":  "", "Height (cm)":  "", "Allow customer reviews?":  "1", "Purchase note":  "", "Sale price":  "", "Regular price":  "", "Categories":  "", "Tags":  "", "Shipping class":  "class10", "Images":  "", "Download limit":  "", "Download expiry days":  "", "Parent":  "", "Grouped products":  "", "Upsells":  "", "Cross-sells":  "", "External URL":  "", "Button text":  "", "Position":  "0", "Download 1 name":  "", "Download 1 URL":  "", "Download 2 name":  "", "Download 2 URL":  ""}
defaultmappingoptions = { 'MapSimpleProducts': True, 'MapVariableProducts': True, 'VariationMapping': Nested, 'CategorySeparator': ' -> ', 'AttributeCollectingMethod': UseAttributeTagProperties}
defaultproductnode = 'product'


class AppState():
    _instance = None
    @staticmethod
    def instance():
        if AppState._instance == None:
            AppState()
        return AppState._instance
    
    def __init__(self):
        if AppState._instance != None:
            raise Exception("Instance is already created.")            
        else:
            AppState._instance = self
        self.mappingoptions = self.mappingconfig['MappingOptions']
        self.mapping = self.mappingconfig['Mapping']

    configtab = 0
    mappingtab = 1
    loadconfigmaptab = 2
    saveconfigmaptab = 3
    loadxmltab = 4
    showproductstab = 5
    editproductstab = 6
    exportproductstab = 7
    showntab = configtab

    mappingconfigchanged = False
    mappingconfigset = False
    configandmappingexported = False
    
    configandmappingloadedfile = ''
    productxmlcontent = ''
    productxmlfilename = ''
    mappingconfig = {"MappingOptions": { "MapSimpleProducts": True, "MapVariableProducts": True, "VariationMapping": "ParentSKU", "CategorySeparator": " -> ", "AttributeCollectingMethod": "UsePropertyList" },"Mapping": { "ProductNode": "product", "Type": "type", "Id": "", "SKU": "sku", "Name": "title", "Featured": "", "Visible": "", "ShortDescription": "", "Description": "short_description", "Stock": "quantity", "Weight": "weight", "Length": "", "Width": "", "Height": "", "Note": "", "SalePrice": "price", "RegularPrice": "regular_price", "Category": "categories", "Tags": "", "ShippingClass": "", "Images": "images", "FeaturedImage": "featured_image", "Option": "", "ParentSKU": "parent_sku", "AttributeList": "color, size, gender", "Attribute": "", "AttributeName": "", "AttributeValue": "", "AttributeVisible": ""}}
    mappingoptions = None
    mapping = None

state = AppState.instance()

ext_re = r"\.\w+$"
extdot_re = r"\.(?=\w+$)"
file_re = r"^.*\.[A-Z,a-z,0-9]+$"
url_re = r"((http[s]?):\/\/)?([^:\/\s]+)((\/\w+\/)*\/)([\w\-\.]+[^#?\s]+)(.*)?(#[\w\-]+)?"
file_url_re = r"((http[s]?):\/\/)?([^:\/\s\"<>(\\n)]+)((\/\w{3}\/)*\/)([\w\-\.]+[^#?\s\"<>(\\n)]+)([^:\/\s\"<>(\\n)].*)?(#[\w\-]+)?"

exlessbase = lambda path: os.path.basename(os.path.splitext(path)[0])

def write_json(source_dict, filename = 'new_json.json', overwrite = False, v= False):
    if not source_dict:
        raise Exception("Empty input")

    text = json.dumps(source_dict, ensure_ascii = False, indent = 2)
    output_file = filename
    i = 0
    if not overwrite:
        while True:
            i += 1
            if os.path.exists(output_file):
                sch = re.search(extdot_re, output_file)
                if sch:
                    output_file = re.sub(extdot_re, f"{i}.", filename)
                else:
                    output_file = f"{filename}{i}"  
            else: 
                break
        
    with open(output_file, 'w', encoding = 'utf8') as file:
        if v: print(f"Writing JSON file: {output_file}")
        file.write(text)
        if v: print("-- Done --")
        return output_file

def read_json(filename, encoding = 'utf8'):
    with codecs.open(filename, 'r', encoding) as file:
        json_ = json.load(file)    
        return json_
    
def createcommalist(valuelist, sep = ',', wrap = False, spaced = True):
    if valuelist:
        valuestr = ""
        sep += ' ' if spaced else ''
        for value in valuelist:
            value = value.strip()
            if value:  
                valuestr += value.strip() + sep
        return f'"{valuestr[:-len(sep)]}"' if wrap else valuestr[:-len(sep)]

def parsecommalist(commalist, sep = ','):
    if commalist:
        parsedlist = commalist.split(sep)
        i = 0
        for value in parsedlist:
            parsedlist[i] = value.strip()
            i += 1
        return parsedlist

def writetofile(content, filepath, overwrite= True, append= False):
    if content and filepath:
        dirname = os.path.dirname(filepath)
        os.makedirs(dirname, exist_ok= True)
        with open(filepath, 'w', encoding = 'utf8') as file:
            file.write(content)
        return filepath
    return False

##### CSV #####
  
def wcsv(header, data, filepath, quoting = csv.QUOTE_MINIMAL, v= False):
    if data:
        islist = isinstance(data, list)
        if islist:
            datadict = isinstance(data[0], dict)
        else:
            datadict = isinstance(data, dict)

        with open(filepath, 'w', newline="") as csvfile:
            if  datadict:
                writer = csv.DictWriter(csvfile, fieldnames = header, quoting=csv.QUOTE_MINIMAL, escapechar=' ')
                writer.writeheader()
            else:
                writer = csv.writer(csvfile, quoting = quoting, escapechar=' ')
                writer.writerow(header)
            if islist:  
                writer.writerows(data)
            else :
                writer.writerow(data)
            return filepath
    else:
        errormessage = "wcsv error: data is empty."
        if v: print(errormessage)

def getfilecontent(filepath, v= False):
    if v: print(f"Reading file: {filepath}")
    if os.path.isfile(filepath):
        with open(filepath, 'r') as xfile:
            data = xfile.read()
            if v: print(f"Reading done.\nLength of read data is {len(data)} bytes.")
            return data
    else:
        if v: print('File does not exit.')

###### XML ######

propertyvalue = lambda item, tag = "":(item.get(tag) if tag else item.text) if item else ""

def xmltest(items):
    if not items:
        print("No items")
        return
    else:
        print(f"Total items: {len(items)}\nSample:\n {items[round(len(items)/2)]}")

elre = r"(?<=<>).*(?=<\/>)"

def xmlviewfile(file, element, attr = ""):
    items = xmlcallfile(file, element)
    print(f"All items: {len(items)}\n")
    for i in range(len(items)):
        val = items[i].text if not attr else items[i].get(attr)
        print(f"Item {i:<5}->   {val:<10}")
    return items

def xmlview(content, element, attr = ""):
    items = xmlcall(content, element)
    print(f"All items: {len(items)}\n")
    for i in range(len(items)):
        val = items[i].text if not attr else items[i].get(attr)
        print(f"Item {i:<5}->   {val:<10}")
    return items

def xmlviewandeditfile(file, element, value= None, numbers = False, savefile = None, v= False):
    if os.path.isfile(file):
        if not savefile:
            savefile = f"{os.path.splitext(file)[0]}-edited.xml"
        filecontent = getfilecontent(file)
        return xmlviewandedit(filecontent, element, value, numbers, savefile, v)

def xmlviewandedit(filecontent, element, value= None, numbers = False, savefile = None, v= False):
    count = subs = 0
    target = r'(?<=<' + element + r'>).*(?=<\/'+ element + '>)'

    if value:
        newtxt = filecontent
        if v: print(f"{'Updating values in:':>19} <{element}>\n{'Reevaluation:':>19} {value} (x: old value)\n")
    else:
        if v: print(f"Values in <{element}>:\n")

    mchs = re.findall(target, filecontent)
    if mchs:
        matches = set(mchs) if value else mchs
        count = len(matches)
        i = 0
        for mch in matches:
            if value:
                oldval = mch if not numbers else ast.literal_eval(mch)
                newval = eval(value, {"x": oldval})
                thistarget = target.replace(".*", mch)
                sub = re.subn(thistarget, str(newval), newtxt)
                newtxt = sub[0]
                subs += sub[1]
                if v: print( f"{i:<4}- {str(mch):<15} {'->':<15} {str(newval):<15}Subs {sub[1]}")
            else:
                if v: print(f"{i:<4}- {mch:<10}")
            i += 1
    if value and savefile:
        with open(savefile, 'w') as wfile:
            wfile.write(newtxt)
            if v: print(f"\nTotal matches: {count}     Values changed: {subs}\n")
        return savefile
    else:
        if v: print(f"\nTotal values: {count}\n")
        return newtxt

def xmlcallfile(filepath, element, callback= None, v=False):
    if os.path.isfile(filepath):
        content = getfilecontent(filepath)
        return xmlcall(content, element, callback)
    else:
        if v: print("xmlcall error: provided file path isn't there.")
        return None
    
def xmlcall(filecontent, element, callback = None, v= False):
    if filecontent:
        if v: print(f"Scraping xml from the file.")
        xml_data = BeautifulSoup(filecontent, "xml")
        if xml_data == None:
            if v: print(f"xmlcall error: BS4 counldn't scrape xml data from the file. Data is None.")
            return
        if len(xml_data) == 0:
            if v: print("xmlcall error: BS4 returned empty data. len(xml_data) is 0.")
            return

        items = xml_data.find_all(element)
        if items == None:
            if v: print(f"xmlcall error: BS4 counldn't read requested items with this node name: {element}")
            return
        if len(items) == 0:
            if v: print(f"xmlcall error: BS4 returned empty result for requested items with this node name: {element}")
            return
        if v: print(f"Total items: {len(items)}")
        return callback(items) if callback else items

#### wc product tools

def wcloadmappingconfigfile(mappingconfigjsonfile, v= False):
    loadedmappingconfig = read_json(mappingconfigjsonfile)
    if not loadedmappingconfig:
        if v: print("wcloadmap error. the options json file wasn't read.")
        return
    return loadedmappingconfig

def wcexportmappingconfigfile(mappingconfig, filepath= "", v= False):
    options = mappingconfig['MappingOptions']
    mapping = mappingconfig['Mapping']
    if not options['VariationMapping']: options['VariationMapping'] = defaultmappingoptions['VariationMapping']
    if not options['CategorySeparator']: options['CategorySeparator'] = defaultmappingoptions['CategorySeparator']
    if not options['AttributeCollectingMethod']: options['AttributeCollectingMethod'] = defaultmappingoptions['AttributeCollectingMethod']
    if not filepath:
        filepath = "wc-xml-mapping.json"
        overwrite = False
    else:
        overwrite = True
    if mapping and options:
        if 'write_json' in globals():
            return write_json(mappingconfig, filepath, overwrite)
        else:
            if v: print("write_json is undefined.")
    else:
        if v: print("Mapping is undefined.")

def wccats(values, separator):
    valuelist = values.split(separator)
    if valuelist:
        return createcommalist(valuelist, sep = ' > ')

def finditem(somelist: list, comparingmethod, deleteiffound = False):
    for index in range(len(somelist)):
        if comparingmethod(somelist[index]):
            if deleteiffound:
                return somelist.pop(index)
            else:
                return somelist[index]
    return None

def finditems(somelist: list, comparingmethod, deleteiffound = False):
    foundlist = []
    deleted = 0
    for index in range(len(somelist)):
        if not somelist:
            break
        item = somelist[index - deleted]
        if comparingmethod(item):
            foundlist.append(item)
            if deleteiffound:
                somelist.pop(index - deleted)
                deleted += 1
    return foundlist

def wcmapproductinfo(product, mappingoptions, mapping, defaultvalues: dict, v= False):
    separator = mappingoptions["CategorySeparator"]
    useattributetagproperties = mappingoptions["AttributeCollectingMethod"] == UseAttributeTagProperties
    productmapping = defaultvalues.copy()
    productsku = product.find(mapping["SKU"]).text
    productname = product.find(mapping["Name"]).text
    if not logmaker(): return
    if product.product_parentsku != None:
        productmapping["Parent"] = product.product_parentsku
    productmapping["Name"] = productname
    productmapping["Type"] = product.product_type
    productmapping["Description"] = product.find(mapping["Description"]).text
    productmapping["Short description"] = propertyvalue(product.find(mapping["ShortDescription"])) if mapping["ShortDescription"] else ""
    imgitems = product.find_all(mapping["Images"])
    imglist = list(map(propertyvalue, imgitems))
    if mapping["FeaturedImage"]:
        featuredimage = product.find(mapping["FeaturedImage"]).text
        if featuredimage and not featuredimage in imglist:
            imglist.insert(0, featuredimage)
    productmapping["Images"]  = createcommalist(imglist)
    productmapping["SKU"] = productsku
    productmapping["Regular price"] = product.find(mapping["RegularPrice"]).text
    productmapping["Sale price"] = propertyvalue(product.find(mapping["SalePrice"])) if mapping["SalePrice"] else ""
    if product.product_type == 'simple' or product.product_type == 'variable':
        cats= product.find(mapping["Category"]).text
        if separator.strip() != '>':
            cats = wccats(cats, separator)
        productmapping['Categories'] = cats
        if mapping["Tags"]: productmapping["Tags"] = propertyvalue(product.find(mapping["Tags"]))
    if mapping["Stock"]:
        stock = propertyvalue(product.find(mapping["Stock"]))
        productmapping["Stock"] = stock
        productmapping["In stock?"] = "0" if stock == "0" or stock == "" else "1"
    else:
        productmapping["In stock?"] = "1"
    if product.product_type == 'variation':
        product[f"Is featured?"] = "0"
    if mapping["Height"]: productmapping["Height (cm)"] = propertyvalue(product.find(mapping["Height"]))           
    if mapping["Width"]: productmapping["Width (cm)"] = propertyvalue(product.find(mapping["Width"])) 
    if mapping["Length"]: productmapping["Length (cm)"] = propertyvalue(product.find(mapping["Length"])) 
    if mapping["Weight"]: productmapping["Weight (kg)"] = propertyvalue(product.find(mapping["Weight"]))
    if mapping["ShippingClass"]: productmapping["Shipping class"] = propertyvalue(product.find(mapping["ShippingClass"]))
    #Set the parent product attributes and their values from variations
    if product.product_attributes == None:
        if product.product_type == 'variable':
            product.product_attributes = product.parent_attributes
        else:
            product.product_attributes = parsecommalist(mapping["AttributeList"]) if not useattributetagproperties else product.find_all(mapping["Attribute"])
    attrstr = ""
    if product.product_attributes:
        j = 1
        for attr in product.product_attributes:
            if product.product_type == 'variable':
                attrname = attr
                attrval = createcommalist(product.product_attributes[attr], spaced= False)
            elif useattributetagproperties:
                attrname = attr.get(mapping["AttributeName"])
                attrval = attr.get(mapping["AttributeValue"])
            else:
                attrname = attr
                attrval = product.find(attr).text
            if attrname and attrval:
                attrstr += f'{attrname} [{attrval.replace(",", ", ")}] '
                productmapping[f"Attribute {j} name"] = attrname
                productmapping[f"Attribute {j} value(s)"] = attrval
                productmapping[f"Attribute {j} global"] = "1"
                if product.product_type == 'variable':
                    productmapping[f"Attribute {j} visible"] = "1"
                elif product.product_parent:
                    if not attrname in product.product_parent.parent_attributes:
                        product.product_parent.parent_attributes[attrname] = set()
                    product.product_parent.parent_attributes[attrname].add(attrval)
                j += 1
    attrstr = "Attributes: " + attrstr if attrstr else ""
    if v: print(f"Product mapped:  ->  Type: {product.product_type.capitalize().ljust(10)} SKU: {productsku.ljust(25)} Name: {productname.ljust(75)} {attrstr}") 
    return productmapping

## mapping with shared sku
# ✓ loop and find matches of sku property
# ✓ if sku matches not found. add to simple list
# ✓ map all variations except for the the first match. 
# ✓ map the first as parent variable product
# ✓ map simple list 

def wcmapwithsharedsku(products, mappingconfig: dict, defaultvalues: dict, v= False):
    if not mappingconfig:
        if v: print("Keymap is undefined or empty")
        return
    if not products:
        if v: print("Items is undefined or empty")
        return
    export = []
    options = mappingconfig["MappingOptions"]
    mapping  = mappingconfig["Mapping"]
    mapvariables = options["MapVariableProducts"]
    mapsimples = options["MapSimpleProducts"]
    useattributetagproperties = options["AttributeCollectingMethod"] == UseAttributeTagProperties
    simplesproducts = []
    mapcount = 0
    for i in range(len(products)):
        if not products:
            break
        possibleparent = products.pop(0)
        productsku = possibleparent.find(mapping["SKU"]).text
        #finding nemos.
        getvariationsmethod = lambda possiblevariation: possiblevariation.find(mapping['SKU']).text == productsku
        variationsmatches = finditems(products, getvariationsmethod, deleteiffound= True)
        if variationsmatches:
            possibleparent.product_type = 'variable'
            if not mapvariables:
                continue
            if possibleparent.parent_attributes == None:
                possibleparent.parent_attributes = {}
            j = 1
            for variation in variationsmatches:
                if mapping["AttributeList"] and not useattributetagproperties:
                    product_attributes = parsecommalist(mapping["AttributeList"])
                else:
                    product_attributes = variation.find_all(mapping["Attribute"])
                if not product_attributes:
                    if v: print(f'wcmapwithsharedsku error: can not fetch variation attribute properties from the key mappings.\nVariations will be mapped as simple products\nAttributeList {mappingconfig["AttributeList"]} Attribute {mappingconfig["Attribute"]}')
                    continue
                variation.find(mapping["SKU"]).string = f"{productsku}-V0{j}"
                variation.product_type = 'variation'
                variation.product_parent = possibleparent
                variation.product_parentsku = productsku
                variation.product_attributes = product_attributes
                addvariation = wcmapproductinfo(variation, options, mapping, defaultvalues)
                export.append(addvariation)
                mapcount += 1
                j += 1
            mapthedad = wcmapproductinfo(possibleparent, options, mapping, defaultvalues)
            export.append(mapthedad)
            mapcount += 1
        else:
            possibleparent.product_type = 'simple'
            simplesproducts.append(possibleparent)
    if mapsimples:
        for simpleproduct in simplesproducts:
            if simpleproduct.product_type == 'simple':
                simplemapping = wcmapproductinfo(simpleproduct, options, mapping, defaultvalues)
                export.append(simplemapping)
                mapcount += 1
    if v: print(f"Total mappings: {mapcount}")
    return export

def wcmapwithparentsku(products, mappingconfig: dict, defaultvalues: dict, v= False):
    if not mappingconfig:
        if v: print("Mapping Config is undefined or empty")
        return
    if not products:
        if v: print("Items is undefined or empty")
        return
    export = []
    mappingoptions = mappingconfig["MappingOptions"]
    mapping  = mappingconfig["Mapping"]
    mapvariables = mappingoptions["MapVariableProducts"]
    mapsimples = mappingoptions["MapSimpleProducts"]
    useattributetagproperties = mappingoptions["AttributeCollectingMethod"] == UseAttributeTagProperties
    simplesandvariables = products.copy()
    mapcount = 0
    i = 0
    for product in products:
        if mapvariables and not mapping["ParentSKU"]:
            if v: print("Mapping doesn't have required ParentSKU key.")
            return
        possibleparentsku = product.find(mapping["ParentSKU"]).text
        productsku = product.find(mapping["SKU"]).text
        if possibleparentsku and possibleparentsku != productsku:
            #finding nemo.
            getparentmethod = lambda possibleparent: possibleparent.find(mapping['SKU']).text == possibleparentsku
            parent = finditem(simplesandvariables, getparentmethod, deleteiffound= False)
            if parent:
                if not useattributetagproperties:
                    product_attributes = parsecommalist(mapping["AttributeList"])
                else:
                    product_attributes = product.find_all(mapping["Attribute"])
                if not product_attributes:
                    if v: print(f'wcmapwithparentsku error: can not fetch attribute properties from the key mappings.\nVariations will be mapped as simple products\nAttributeList {mappingconfig["AttributeList"]} Attribute {mappingconfig["Attribute"]}')
                    continue
                parent.product_type = 'variable'
                if parent.parent_attributes == None:
                    parent.parent_attributes = {}
                product.product_type = 'variation'
                product.product_parent = parent
                product.product_parentsku = possibleparentsku
                product.product_attributes = product_attributes
                singlevariation = wcmapproductinfo(product, mappingoptions, mapping, defaultvalues)
                export.append(singlevariation)
                mapcount += 1
                #remove variation from simpleandvaiable list.
                simplesandvariables.remove(product)
        i += 1
    for product in simplesandvariables:
        issimple = product.product_type != 'variable'
        if issimple:
            product.product_type = 'simple'
        domap = (not issimple and mapvariables) or (issimple and mapsimples)
        if domap:
            productmapping = wcmapproductinfo(product, mappingoptions, mapping, defaultvalues)
            export.append(productmapping)
            mapcount += 1
    if v: print(f"Total mappings: {mapcount}")
    return export

def wcmapnested(products, mappingconfig: dict, defaultvalues: dict, v= False):
    if not mappingconfig:
        if v: print("wcmapnested error: Keymap is undefined or empty")
        return
    if not products:
        if v: 
            print("wcmapnested error: wcmapnestedItems is undefined or empty")
            print(f"Items type: {type(products)} Len:{len(products) if products != None else 'items is None.'}\n")
        return
    export = []
    mappingoptions = mappingconfig["MappingOptions"]
    mapping = mappingconfig["Mapping"]
    mapvariables = mappingoptions["MapVariableProducts"]
    mapsimples = mappingoptions["MapSimpleProducts"]
    mapcount = 0
    for product in products:
        options = product.find_all(mapping["Option"])
        variable = len(options) > 1
        #shared props between variable/simple/variation
        name = product.find(mapping["Name"]).text
        desc = product.find(mapping["Description"]).text
        shortdesc = propertyvalue(product.find(mapping["ShortDescription"])) if mapping["ShortDescription"] else ""
        catsraw = product.find(mapping["Category"]).text
        cats = wccats(catsraw, mappingoptions["CategorySeparator"])
        imgitems = product.find_all(mapping["Images"])
        imgs = list(map(propertyvalue, imgitems))
        imgstr = createcommalist(imgs)
        if variable and mapvariables:
            parentproductsku = f"{options[0].find(mapping['SKU']).text[:-1]}00"
            parentattrs = {}
            parentproduct = defaultvalues.copy()
            parentproduct["Type"] = 'variable'
            parentproduct["SKU"] = parentproductsku  
            parentproduct["Name"] = name
            parentproduct["Images"] = imgstr
            parentproduct["Description"] = desc
            parentproduct["Short description"] = shortdesc
            parentproduct['Categories'] = cats
            if mapping["Tags"]:
                parentproduct["Tags"] = propertyvalue(product.find(mapping["Tags"]))
            export.append(parentproduct)
            mapcount += 1
            if v: print(f"Variable:   ->  {parentproductsku}\n")
        #add simple product or nested variations.
        for option in options:
            singlevariation = defaultvalues.copy()
            #Set shared props for simple product/variation.
            singlevariation["SKU"] = option.find(mapping["SKU"]).text
            singlevariation["Regular price"] = option.find(mapping["RegularPrice"]).text
            singlevariation["Sale price"] = propertyvalue(option.find(mapping["SalePrice"])) if mapping["SalePrice"] else ""
            if mapping["Stock"]:
                stock = propertyvalue(option.find(mapping["Stock"]))
                singlevariation["Stock"] = stock
                singlevariation["In stock?"] = "0" if stock == "0" or stock == "" else "1"
            if mapping["Height"]:
                singlevariation["Height (cm)"] = propertyvalue(option.find(mapping["Height"]))           
            if mapping["Width"]:
                singlevariation["Width (cm)"] = propertyvalue(option.find(mapping["Width"])) 
            if mapping["Length"]:
                singlevariation["Length (cm)"] = propertyvalue(option.find(mapping["Length"])) 
            if mapping["Weight"]:
                singlevariation["Weight (kg)"] = propertyvalue(option.find(mapping["Weight"]))
            if mapping["ShippingClass"]:
                singlevariation["Shipping class"] = propertyvalue(option.find(mapping["ShippingClass"]))
            #if parent is variable add as a variation.  
            if variable:
                if mapvariables:
                    singlevariation["Type"] = 'variation'
                    singlevariation["Parent"] = parentproductsku
                    attrset = option.find_all(mapping["Attribute"])
                    j = 1
                    namevariation = ""
                    for attr in attrset:
                        attrname = attr.get(mapping["AttributeName"])
                        attrval = attr.get(mapping["AttributeValue"])
                        if v: print(f"Variation:  ->  attr: {attrname}  val: {attrval}\n")
                        singlevariation[f"Attribute {j} name"] = attrname
                        singlevariation[f"Attribute {j} value(s)"] = attrval
                        singlevariation[f"Attribute {j} global"] = "1"
                        singlevariation[f"Is featured?"] = "0"
                        if not attrname in parentattrs:
                            parentattrs[attrname] = set()          
                        parentattrs[attrname].add(attrval)
                        if j == 1:
                            namevariation = attrval
                        else:
                            namevariation += f" {attrval}"
                        j += 1
                    singlevariation["Name"] = f"{name} {namevariation}"
            #add as simple product
            elif mapsimples:
                singlevariation["Type"] = 'simple'
                singlevariation["Images"] = imgstr
                singlevariation["Short description"] = shortdesc
                singlevariation["Name"] = name
                singlevariation['Categories'] = cats
                singlevariation["Description"] = desc
                if mapping["Tags"]:
                    singlevariation["Tags"] = propertyvalue(option.find(mapping["Tags"]))
            export.append(singlevariation)
            mapcount += 1
        #Set the parent product attributes and their values from variations
        if variable:
            if mapvariables:
                i = 1
                for attrkey in parentattrs:
                    if v: print(f"Attribute Values:  {attrkey}  ->  {str(parentattrs[attrkey])}\n")
                    parentproduct[f"Attribute {i} name"] = attrkey
                    parentproduct[f"Attribute {i} value(s)"] = createcommalist(parentattrs[attrkey])
                    parentproduct[f"Attribute {i} visible"] = "1"
                    parentproduct[f"Attribute {i} global"] = "1"
        elif mapsimples:
            if v: print(f"Simple:   ->  {singlevariation['SKU']}\n")
    if v: print(f"Total mappings: {mapcount}")
    return export
    
def wcmapxmlfile(xmlpath, mappingconfig, defaultvalues):
    csvoutput = csvoutput = f"{exlessbase(xmlpath)}.csv"
    return wcmapxml(xmlfile= xmlpath, csvoutput= csvoutput, mappingconfig= mappingconfig, defaultvalues= defaultvalues)

def wcmapxmlstring(xmlstring, csvoutput, mappingconfig, defaultvalues):
    return wcmapxml(xmlsource= xmlstring, csvoutput= csvoutput, mappingconfig= mappingconfig, defaultvalues= defaultvalues)

def wcmapxml(xmlsource= '', xmlfile= '', csvoutput= '', mappingconfig = None, defaultvalues= None):
    if not mappingconfig:
        print("wcmapxml error: Keymap is undefined or empty.")
    try:
        scrapenode = mappingconfig["Mapping"]["ProductNode"]
    except:
        scrapenode = defaultproductnode
    variationmapping = mappingconfig["MappingOptions"]["VariationMapping"]
    
    if variationmapping == SharedSKU:
        wcmapfunc = wcmapwithsharedsku
    elif variationmapping == ParentSKU:
        parentsku = mappingconfig['Mapping']['ParentSKU']
        if not parentsku:
            mappingconfig["MappingOptions"]["MapVariableProducts"] = False
            print("ParentSKU key mapping isn't configured. variable products will be mapped as simple products.")
        wcmapfunc = wcmapwithparentsku
    elif variationmapping == Nested:
        wcmapfunc = wcmapnested
    else:
        mappingconfig["MappingOptions"]["MapVariableProducts"] = False
        wcmapfunc = wcmapnested
    wcproc = lambda items: wcmapfunc(items, mappingconfig, defaultvalues)
    importjob = xmlcall(xmlsource, scrapenode, wcproc) if xmlsource else xmlcallfile(xmlfile, scrapenode, wcproc)
    return wcsv(wocommerce_header, importjob, csvoutput, quoting = csv.QUOTE_ALL)

def wcitemtesting(items, mappingfile= None, mappings= None):
    print(f"Woocommerce Item Testing\nRecieved items count: {len(items)}")
    i = 0

    for item in items:
        print(items[i].product_type == None)
        if i == 1:
            item.product_type = "Variable"
            item.product_attributes = {"color": set()}
            item.product_attributes["color"].add('yellow')
        i += 1

    items[1].product_attributes["color"].add("red")
    if not 'size' in items[1].product_attributes:
        items[1].product_attributes['size'] = set()
        items[1].product_attributes['size'].add('XLLL')
        items[1].product_attributes['size'].add('L')
        items[1].product_attributes['size'].add('S')
        items[1].product_attributes['size'].add('L')

    if items[1].product_attributes:
        print(items[1].product_attributes)
    print(getattr(items[1], 'product_type', False))
    print(items[1].product_type == None)

def wcpropertyview(content= "", filepath= "", pricenode = "", salepricenode = "", currencynode = ""):
    if content:
        if pricenode:
            xmlviewandedit(filepath, pricenode)
        if salepricenode:
            xmlviewandedit(filepath, salepricenode)
        if currencynode:
            xmlviewandedit(filepath, currencynode)

    elif filepath:
        if pricenode:
            xmlviewandedit(filepath, pricenode)
        if salepricenode:
            xmlviewandedit(filepath, salepricenode)
        if currencynode:
            xmlviewandedit(filepath, currencynode)
    return

def wceditprices(content= "", pricenode = "", salepricenode = "",  pricechange = "", currrate = "", v= False):
    priceupdateoperation = ''
    dopriceupdate = pricechange or currrate
    if dopriceupdate:
        pricechange = pricechange.strip()
        if (pricenode and pricechange) or currrate:
            if pricechange[0] != '+' and pricechange[0] != '-':
                raise Exception("In order to complete price update the price change must be preceeded with a sign.")
                
            sign = pricechange[0]
            pricechange = pricechange[1:]
            amount = f"x*{float(pricechange.replace('%',''))/100}" if '%' in pricechange else pricechange
            priceupdate = sign + amount if amount else ""
            priceupdate = f"x {priceupdate}" if not currrate else f"(x {priceupdate})*{currrate}"
            if v: print(f"{'Requested price change:':>18} {priceupdate}")
            priceupdateoperation = f"round(({priceupdate}), 2)"

        #newfilenamesuffix = f" {sign + pricechange if pricechange else ''}{' x' + currrate if currrate else ''}"
        #splittedfilename = exlessbase(filepath)
        #newfilename = saveto + splittedfilename + newfilenamesuffix + ".xml"
    if priceupdateoperation:
        if pricenode:
            if v: print(f"Updating price values. Price property xml node name: {pricenode}")
            returnedcontent = xmlviewandedit(filecontent= content, element= pricenode, value = priceupdateoperation, numbers = True)
        else:
            returnedcontent = content         
        if salepricenode:
            if v: print(f"Updating sale price values. Sale price property xml node name: {salepricenode}")
            returnedcontent = xmlviewandedit(filecontent= returnedcontent, element= salepricenode, value = priceupdateoperation, numbers = True)
        return returnedcontent

    #docurrcodeupdate: xmledit(filepath, currnode, value = f"'{currunit}'", numbers = False, savefile = newfilename)          
    
def fixmissingSku():
  return

###### 

def setappconfigfrominput():
    state.mappingoptions['MapSimpleProducts'] = mapsimple_checkbox.value  
    state.mappingoptions['MapVariableProducts'] = mapvariable_checkbox.value 
    state.mappingoptions['VariationMapping'] = variationmapping_dropdown.value
    state.mappingoptions['CategorySeparator'] = categoryseparator_text.value
    state.mappingoptions['AttributeCollectingMethod'] = attributemethod_dropdown.value 

def setwcmappingfrominput():
    state.mapping['ProductNode']= productnode_t21_text.value if productnode_t21_text.value  else defaultproductnode
    state.mapping["Type"]  = type_t22_text.value
    state.mapping["Id"]  = id_t23_text.value                      
    state.mapping["SKU"] = sku_t24_text.value                               
    state.mapping["Name"] = name_t25_text.value                                
    state.mapping["Featured"] = featured_t26_text.value                                  
    state.mapping["Visible"] = visible_t27_text.value                                   
    state.mapping["ShortDescription"] = shortdesc_t28_text.value                    
    state.mapping["Description"] = description_t29_text.value                      
    state.mapping["Stock"] = stock_t210_text.value                             
    state.mapping["Weight"] = weight_t211_text.value                              
    state.mapping["Length"] = length_t212_text.value                                   
    state.mapping["Width"] = width_t213_text.value                                  
    state.mapping["Height"] = height_t214_text.value                                     
    state.mapping["Note"] = note_t215_text.value                            
    state.mapping["SalePrice"] = saleprice_t221_text.value                 
    state.mapping["RegularPrice"] = regularprice_t222_text.value                   
    state.mapping["Category"] = category_t223_text.value                       
    state.mapping["Tags"] = tags_t2214_text.value                                
    state.mapping["ShippingClass"] = shippingclass_t224_text.value                            
    state.mapping["Images"] = images_t226_text.value   
    state.mapping["FeaturedImage"] = featured_t26_text.value   
    state.mapping["Option"] = option_t227_text.value
    state.mapping["ParentSKU"] = parentsku_t228_text.value
    state.mapping["AttributeList"] = attibutelist_t229_text.value                        
    state.mapping["Attribute"] = attribute_t2210_text.value                           
    state.mapping["AttributeName"] = attributename_t2211_text.value                      
    state.mapping["AttributeValue"] = attributevalue_t2212_text.value                        
    state.mapping["AttributeVisible"] = attributevisible_t2213_text.value
    
def setuploadedmappingconfig(loadedconfig):
    state.mappingoptions = loadedconfig['MappingOptions']
    state.mappingconfig['MappingOptions'] = state.mappingoptions

    mapsimple_checkbox.value = state.mappingoptions['MapSimpleProducts'] 
    mapvariable_checkbox.value = state.mappingoptions['MapVariableProducts'] 
    variationmapping_dropdown.value = state.mappingoptions['VariationMapping'] 
    categoryseparator_text.value = state.mappingoptions['CategorySeparator'] 
    attributemethod_dropdown.value = state.mappingoptions['AttributeCollectingMethod']

    state.mapping = loadedconfig['Mapping']
    state.mappingconfig['Mapping'] = state.mapping
    productnode_t21_text.value = state.mapping['ProductNode']
    type_t22_text.value = state.mapping["Type"]  
    id_t23_text.value = state.mapping["Id"]  
    sku_t24_text.value = state.mapping["SKU"] 
    name_t25_text.value = state.mapping["Name"] 
    featured_t26_text.value = state.mapping["Featured"] 
    visible_t27_text.value = state.mapping["Visible"] 
    shortdesc_t28_text.value = state.mapping["ShortDescription"] 
    description_t29_text.value = state.mapping["Description"] 
    stock_t210_text.value = state.mapping["Stock"] 
    weight_t211_text.value = state.mapping["Weight"] 
    length_t212_text.value = state.mapping["Length"] 
    width_t213_text.value = state.mapping["Width"] 
    height_t214_text.value = state.mapping["Height"] 
    note_t215_text.value = state.mapping["Note"] 
    saleprice_t221_text.value = state.mapping["SalePrice"] 
    regularprice_t222_text.value = state.mapping["RegularPrice"] 
    category_t223_text.value = state.mapping["Category"] 
    tags_t2214_text.value = state.mapping["Tags"] 
    shippingclass_t224_text.value = state.mapping["ShippingClass"] 
    images_t226_text.value = state.mapping["Images"] 
    featured_t26_text.value = state.mapping["FeaturedImage"] 
    option_t227_text.value = state.mapping["Option"] 
    parentsku_t228_text.value = state.mapping["ParentSKU"] 
    attibutelist_t229_text.value = state.mapping["AttributeList"] 
    attribute_t2210_text.value = state.mapping["Attribute"] 
    attributename_t2211_text.value = state.mapping["AttributeName"] 
    attributevalue_t2212_text.value = state.mapping["AttributeValue"] 
    attributevisible_t2213_text.value = state.mapping["AttributeVisible"] 

#### UI
#### layout guide
# center horizontally layout.align_items = 'center' for flex_flow column
# center vertically  layout.justify_content = 'center'  for flex_flow column

### general layout and style

layout_15 = {'width':'auto', 'flex':'15 1 0%', 'align_items':"center"}
layout_10 = {'width':'auto', 'flex':'10 1 0%', 'align_items':"center"}
layout_8 =  {'width':'auto', 'flex':'8 1 0%', 'align_items':"center"}
layout_7 =  {'width':'auto', 'flex':'7 1 0%', 'align_items':"center"}
layout_5 =  {'width':'auto', 'flex':'5 1 0%', 'align_items':"center"}
layout_4 =  {'width':'auto', 'flex':'4 1 0%', 'align_items':"center"}
layout_3 =  {'width':'auto', 'flex':'3 1 0%', 'align_items':"center"}
layout_2 =  {'width':'auto', 'flex':'2 1 0%', 'align_items':"center"}
layout_1 =  {'width':'auto', 'flex':'1 1 0%', 'align_items':"center"}
layout_1E = {'width':'auto', 'flex':'1 1 0%', 'align_items':"flex-start"}

collayout14 = {'width':'%25', 'height':'100%','flex':'3 1 0%'}
collayout34 = {'width':'%75', 'height':'100%','flex':'9 1 0%'}
collayout13 = {'width':'%33','flex':'2 1 0%', 'align_items':"center"}
collayout12 = {'width':'%50','flex':'1 1 0%', 'align_items':"center"}

empty_label_1 = widgets.Label(value = "", layout = layout_1)
empty_label_2 = widgets.Label(value = "", layout = layout_2)
empty_label_3 = widgets.Label(value = "", layout = layout_3)
empty_label_4 = widgets.Label(value = "", layout = layout_4)
empty_label_5 = widgets.Label(value = "", layout = layout_5)

whitespace1 = ' '
whitespace2 = '  '
whitespace3 = '   '
whitespace4 = '    '
whitespace5 = '     '

### custom style and layout

font_family = 'Tahoma'

row1_layout = {'flex_flow':'row', 'align_items':'center', 'width':'90%', 'height':'130px', 'justify_content':'center'}
row2_layout = {'flex_flow':'row', 'align_items':'center', 'width':'90%', 'height':'620px', 'justify_content':'center'}

mainbox_layout = { 'height':'100%', 'width': '100%', 'margin':'0 0 20px 0' , 'border': '1px solid orange', 'display': 'none'}
maincontent_layout = {'width':'100%', 'height':'100%', 'align_content': 'center', 'justify_content': 'flex-start', 'padding': '50px 20px 20px 50px'}

menubtns_style = {'font_family':font_family, 'font_size':'16px', 'font_weight':'600'}
menubtns_layout = {'width':'80%', 'height':'50px', 'margin': '0 0 8px 0'}

input_style = {"description_width":"120px"}
samplestyle = {"padding-left":"200px","padding-right":"200px"}
conflabelstyle = {"padding-left":"150px"}

bodytitlelabel_layout = { 'height': '70px'}
bodytitlelabel_style = {'font_family': font_family, 'font_size': '30px'}
bodybutton_layout = {'width': '200px', 'height': '70px', 'margin':'0 0 40px 0'}
bodystatuslabel_style = {'font_family': font_family, 'font_size': '18px'}

# main row1
### r1v1

logo_label_style = {'font_size': '36px','font_weight':'600','font_family': font_family}

logo = IPython.display.Image('https://niotro.com/files/proddo/assets/proddo.png', width = 193)
logo_image = widgets.Image(value= logo.data, width = 193 )
logo_image.layout.justify_content = 'center'

r1v1_vbox = widgets.VBox([logo_image], layout= collayout14)
r1v1_vbox.layout.justify_items = 'center'
r1v1_vbox.layout.align_items = 'center'

### r1v2

statelabels_style = {'font_family': font_family, 'font_size': '18px'}
r1v2configuration_label = widgets.Label(style= statelabels_style, value= 'Configuration: configuration not set yet.')
r1v2xmlfile_label = widgets.Label(style= statelabels_style, value= 'Products XML file: No XML file loaded')
r1v2save_button = widgets.Button(description= 'Save',  layout= {'width': '150px', 'height': '35px'}, button_style= 'success', disabled= not debugbuttons)

r1v2_vbox = widgets.VBox([r1v2configuration_label, r1v2xmlfile_label, r1v2save_button], layout= collayout34)
r1v2_vbox.layout.justify_content = 'flex-end'
r1v2_vbox.layout.padding = '0 0 10px 0'

row1_hbox = widgets.HBox([r1v1_vbox,r1v2_vbox], layout= row1_layout)

# row 2
### r2v1

configmenu_button = widgets.Button(description= 'Configuration', style= menubtns_style, layout= menubtns_layout)
productmappingmenu_button = widgets.Button(description= 'Product Mapping', style= menubtns_style, layout= menubtns_layout)
loadconfigmenu_button = widgets.Button(description= 'Load Configuration',style= menubtns_style,layout= menubtns_layout)
savemappingconfigmenu_button = widgets.Button(description= 'Save Configuration',style= menubtns_style,layout= menubtns_layout)
savemappingconfigmenu_button.layout.margin = '0 0 20px 0'
loadxmlproductsmenu_button = widgets.Button(description= 'Load XML Products',style= menubtns_style,layout= menubtns_layout)
showproductsmenu_button = widgets.Button(description= 'Show Products', style= menubtns_style, layout= menubtns_layout, disabled= not debugbuttons)
editproductspricesmenu_button = widgets.Button(description= 'Edit Products Prices', style= menubtns_style, layout= menubtns_layout, disabled= not debugbuttons)
exportproductsmenu_button = widgets.Button(description= 'Export Products', style= menubtns_style, layout= menubtns_layout, disabled= not debugbuttons)

r2v1_vbox = widgets.VBox([configmenu_button, productmappingmenu_button, loadconfigmenu_button,
                        savemappingconfigmenu_button, loadxmlproductsmenu_button, showproductsmenu_button,
                        editproductspricesmenu_button, exportproductsmenu_button],
                        layout= collayout14)
r2v1_vbox.layout.justify_content = 'flex-start'
r2v1_vbox.layout.align_items = 'center'

### r2v2
#### main box

t1formrow_layout = {'height':'50px', 'width':'100%', 'justify_content': 'flex-start'}
t1labels_layout = {'width':'35%', 'justify_content': 'flex-end'}
t1labels_style = {'font_family': font_family, 'font_size': '18px'}
t1checkboxs_layout = {'margin':'5px 0 0 10px', 'justify_items': 'flex-start','justify_content': 'flex-start','align_content': 'flex-start',}
t1controls_layout = {'margin':'0 0 0 10px', 'width':'250px', 'justify_items': 'flex-start', 'justify_content': 'flex-start', 'align_content': 'flex-start'}
t1controls_style = {'font_family': font_family, 'font_size': '16px', 'description_width':'0px'}

#### t1

mapsim_label = widgets.Label(value= "Map Simple Products", layout= t1labels_layout, style= t1labels_style)
mapsimple_checkbox = widgets.Checkbox(value= state.mappingoptions['MapSimpleProducts'], layout= t1checkboxs_layout, style= t1controls_style)
t1r1_hbox = widgets.HBox([mapsim_label, mapsimple_checkbox], layout= t1formrow_layout)

mapvar_label = widgets.Label(value= "Map Variable Products", layout= t1labels_layout, style= t1labels_style)
mapvariable_checkbox = widgets.Checkbox(value= state.mappingoptions['MapVariableProducts'], layout= t1checkboxs_layout, style= t1controls_style)
t1r2_hbox = widgets.HBox([mapvar_label, mapvariable_checkbox], layout= t1formrow_layout)

catsep_label = widgets.Label(value= "Category Separator", layout= t1labels_layout, style= t1labels_style)
categoryseparator_text = widgets.Text(value= state.mappingoptions['CategorySeparator'], layout= t1controls_layout, style= t1controls_style)
t1r3_hbox = widgets.HBox([catsep_label, categoryseparator_text],layout= t1formrow_layout)

varstruct_label = widgets.Label(value= "Variation Mapping",layout= t1labels_layout,style= t1labels_style)
variationmapping_dropdown = widgets.Dropdown(value= state.mappingoptions['VariationMapping'] ,options= [('Nested', Nested), ('Parent SKU', ParentSKU), ('Shared SKU', SharedSKU), ('Map as Simple', MapAsSimple)], layout= t1controls_layout, style= t1controls_style)
t1r4_hbox = widgets.HBox([varstruct_label, variationmapping_dropdown], layout= t1formrow_layout)

attrcol_label = widgets.Label(value= "Attribute Collection Method", layout= t1labels_layout, style= t1labels_style)
attributemethod_dropdown = widgets.Dropdown(value= state.mappingoptions['AttributeCollectingMethod'], options= [('Use Property List', UsePropertyList), ('Use Attribute Tag Properties', UseAttributeTagProperties),], layout= t1controls_layout, style= t1controls_style)
t1r5_hbox = widgets.HBox([attrcol_label, attributemethod_dropdown], layout= t1formrow_layout)

t1_vbox = widgets.VBox([t1r1_hbox, t1r2_hbox, t1r3_hbox, t1r4_hbox, t1r5_hbox], layout= maincontent_layout)

#### t2

t2formrows_layout = {'height':'40px', 'width':'100%', 'justify_content': 'flex-start'}
t2labels_layout = {'width':'35%', 'justify_content': 'flex-end'}
t2labels_style = {'font_family': font_family, 'font_size': '18px'}

t2controls_layout = {'margin':'0 0 0 10px', 'width':'250px', 'justify_items': 'flex-start', 'justify_content': 'flex-start', 'align_content': 'flex-start'}
t2controls_style = {'font_family': font_family, 'font_size': '16px', 'description_width':'0px'}
t2v_layout = {'width': '50%',}

# t2v1
# col1

productnode_t21_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['ProductNode'])
t21_hbox = widgets.HBox([widgets.Label(value= 'Product Node', layout= t2labels_layout, style= t2labels_style), productnode_t21_text], layout= t2formrows_layout)

type_t22_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Type'])
t22_hbox = widgets.HBox([widgets.Label(value= 'Type', layout= t2labels_layout, style= t2labels_style), type_t22_text], layout= t2formrows_layout)

id_t23_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Id'])
t23_hbox = widgets.HBox([widgets.Label(value= 'Id', layout= t2labels_layout, style= t2labels_style), id_t23_text], layout= t2formrows_layout)

sku_t24_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['SKU'])
t24_hbox = widgets.HBox([widgets.Label(value= 'SKU', layout= t2labels_layout, style= t2labels_style), sku_t24_text], layout= t2formrows_layout)

name_t25_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Name'])
t25_hbox = widgets.HBox([widgets.Label(value= 'Name', layout= t2labels_layout, style= t2labels_style), name_t25_text], layout= t2formrows_layout)

featured_t26_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Featured'])
t26_hbox = widgets.HBox([widgets.Label(value= 'Featured', layout= t2labels_layout, style= t2labels_style), featured_t26_text], layout= t2formrows_layout)

visible_t27_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Visible'])
t27_hbox = widgets.HBox([widgets.Label(value= 'Visible', layout= t2labels_layout, style= t2labels_style), visible_t27_text], layout= t2formrows_layout)

shortdesc_t28_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['ShortDescription'])
t28_hbox = widgets.HBox([widgets.Label(value= 'Short Description', layout= t2labels_layout, style= t2labels_style), shortdesc_t28_text], layout= t2formrows_layout)

description_t29_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Description'])
t29_hbox = widgets.HBox([widgets.Label(value= 'Description', layout= t2labels_layout, style= t2labels_style), description_t29_text], layout= t2formrows_layout)

stock_t210_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Stock'])
t210_hbox = widgets.HBox([widgets.Label(value= 'Stock', layout= t2labels_layout, style= t2labels_style), stock_t210_text], layout= t2formrows_layout)

weight_t211_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Weight'])
t211_hbox = widgets.HBox([widgets.Label(value= 'Weight', layout= t2labels_layout, style= t2labels_style), weight_t211_text], layout= t2formrows_layout)

length_t212_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Length'])
t212_hbox = widgets.HBox([widgets.Label(value= 'Length', layout= t2labels_layout, style= t2labels_style), length_t212_text], layout= t2formrows_layout)

width_t213_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Width'])
t213_hbox = widgets.HBox([widgets.Label(value= 'Width', layout= t2labels_layout, style= t2labels_style), width_t213_text], layout= t2formrows_layout)

height_t214_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Height'])
t214_hbox = widgets.HBox([widgets.Label(value= 'Height', layout= t2labels_layout, style= t2labels_style), height_t214_text], layout= t2formrows_layout)

note_t215_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Note'])
t215_hbox = widgets.HBox([widgets.Label(value= 'Note', layout= t2labels_layout, style= t2labels_style), note_t215_text], layout= t2formrows_layout)

t2v1_vbox = widgets.VBox([t21_hbox, t22_hbox, t23_hbox, t24_hbox, t25_hbox, t26_hbox, t27_hbox, t28_hbox, t29_hbox, t210_hbox, t211_hbox, t212_hbox, t213_hbox, t214_hbox, t215_hbox], layout= t2v_layout)

#col2

saleprice_t221_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['SalePrice'])
t221_hbox = widgets.HBox([widgets.Label(value= 'Sale Price', layout= t2labels_layout, style= t2labels_style), saleprice_t221_text], layout= t2formrows_layout)

regularprice_t222_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['RegularPrice'])
t222_hbox = widgets.HBox([widgets.Label(value= 'Regular Price', layout= t2labels_layout, style= t2labels_style), regularprice_t222_text], layout= t2formrows_layout)

category_t223_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Category'])
t223_hbox = widgets.HBox([widgets.Label(value= 'Category', layout= t2labels_layout, style= t2labels_style), category_t223_text], layout= t2formrows_layout)

shippingclass_t224_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['ShippingClass'])
t224_hbox = widgets.HBox([widgets.Label(value= 'Shipping Class', layout= t2labels_layout, style= t2labels_style), shippingclass_t224_text], layout= t2formrows_layout)

featuredimage_t225_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['FeaturedImage'])
t225_hbox = widgets.HBox([widgets.Label(value= 'Featured Image', layout= t2labels_layout, style= t2labels_style), featuredimage_t225_text], layout= t2formrows_layout)

images_t226_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Images'])
t226_hbox = widgets.HBox([widgets.Label(value= 'Images', layout= t2labels_layout, style= t2labels_style), images_t226_text], layout= t2formrows_layout)

option_t227_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Option'])
t227_hbox = widgets.HBox([widgets.Label(value= 'Option', layout= t2labels_layout, style= t2labels_style), option_t227_text], layout= t2formrows_layout)

parentsku_t228_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['ParentSKU'])
t228_hbox = widgets.HBox([widgets.Label(value= 'Parent SKU', layout= t2labels_layout, style= t2labels_style), parentsku_t228_text], layout= t2formrows_layout)

attibutelist_t229_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['AttributeList'])
t229_hbox = widgets.HBox([widgets.Label(value= 'Attribute List', layout= t2labels_layout, style= t2labels_style), attibutelist_t229_text], layout= t2formrows_layout)

attribute_t2210_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Attribute'])
t2210_hbox = widgets.HBox([widgets.Label(value= 'Attribute', layout= t2labels_layout, style= t2labels_style), attribute_t2210_text], layout= t2formrows_layout)

attributename_t2211_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['AttributeName'])
t2211_hbox = widgets.HBox([widgets.Label(value= 'Attribute Name', layout= t2labels_layout, style= t2labels_style), attributename_t2211_text], layout= t2formrows_layout)

attributevalue_t2212_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['AttributeValue'])
t2212_hbox = widgets.HBox([widgets.Label(value= 'Attribute Value', layout= t2labels_layout, style= t2labels_style), attributevalue_t2212_text], layout= t2formrows_layout)

attributevisible_t2213_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['AttributeVisible'])
t2213_hbox = widgets.HBox([widgets.Label(value= 'Attribute Visible', layout= t2labels_layout, style= t2labels_style), attributevisible_t2213_text], layout= t2formrows_layout)

tags_t2214_text = widgets.Text(layout= t2controls_layout, style= t2controls_style, value= state.mapping['Tags'])
t2214_hbox = widgets.HBox([widgets.Label(value= 'Tags', layout= t2labels_layout, style= t2labels_style), tags_t2214_text], layout= t2formrows_layout)

t2v2_vbox = widgets.VBox([t221_hbox, t222_hbox, t223_hbox, t224_hbox, t225_hbox, t226_hbox, t227_hbox, t228_hbox, t229_hbox, t210_hbox, t2211_hbox, t2212_hbox, t2213_hbox, t2214_hbox],layout= t2v_layout)
t2_hbox = widgets.HBox([t2v1_vbox, t2v2_vbox], layout= maincontent_layout)
t2_hbox.layout.padding = '15px 50px'

#### t3

t3title_label = widgets.Label(value= 'Load Mapping and Configuration', style= bodytitlelabel_style, layout= bodytitlelabel_layout)
loadmappingconfig_fileupload = widgets.FileUpload(description= 'Upload Mapping File', layout= bodybutton_layout)
loadmappingconfigstatus_label = widgets.Label(value= '', style= bodystatuslabel_style)
t3_vbox = widgets.VBox([t3title_label, loadmappingconfig_fileupload, loadmappingconfigstatus_label], layout= maincontent_layout)

#### t4

t4title_label = widgets.Label(value= 'Save Mapping and Configuration', style= bodytitlelabel_style, layout= bodytitlelabel_layout)
savemappingconfig_button = widgets.Button(description= 'Save Mapping File', layout= bodybutton_layout)
savemappingconfigstatus_label = widgets.Label(value= '', style= bodystatuslabel_style)
t4_vbox = widgets.VBox([t4title_label, savemappingconfig_button, savemappingconfigstatus_label], layout= maincontent_layout)

#### t5

t5title_label = widgets.Label(value= 'Load Products XML', style= bodytitlelabel_style, layout= bodytitlelabel_layout)
loadproductsxml_fileupload = widgets.FileUpload(description= 'Load Products File', layout= bodybutton_layout)
loadproductsxmlstatus_label = widgets.Label(value= '', style= bodystatuslabel_style)
t5_vbox = widgets.VBox([t5title_label, loadproductsxml_fileupload, loadproductsxmlstatus_label], layout= maincontent_layout)

#### t6

producttableoutput = widgets.Output() 
t6_vbox = widgets.VBox([producttableoutput], layout= maincontent_layout)
t6_vbox.layout.padding = '0'

#### t7

t7formrow_layout = {'height':'80px', 'width':'100%', 'justify_content': 'flex-start'}
t7title_label = widgets.Label(value= 'Update Products Prices', style= bodytitlelabel_style, layout= bodytitlelabel_layout)

pricechange_text = widgets.Text(layout= t1controls_layout)
t7r1_hbox = widgets.HBox([widgets.Label(value='Price Change: ', layout= t1labels_layout, style= t1labels_style), pricechange_text], layout= t7formrow_layout)

currencyrate_text = widgets.Text(layout= t1controls_layout)
t7r2_hbox = widgets.HBox([widgets.Label(value='Currency Rate: ', layout= t1labels_layout, style= t1labels_style), currencyrate_text], layout= t7formrow_layout)

editproductsprices_button = widgets.Button(description= 'Edit Product Prices', layout= bodybutton_layout)
exporteditedproducts_button = widgets.Button(description= 'Export Edited Products', layout= bodybutton_layout, disabled= True)
t7r3btns_hbox = widgets.HBox([editproductsprices_button, exporteditedproducts_button])
t7r3btns_hbox.layout.width = '50%'
t7r3btns_hbox.layout.justify_content = 'space-around'
editproductspricesstatus_label = widgets.Label(value='', style= bodystatuslabel_style)
t7r3_vbox = widgets.VBox([t7r3btns_hbox, editproductspricesstatus_label], layout= t7formrow_layout)
t7r3_vbox.layout.align_items = 'center'
t7r3_vbox.layout.height = '200px'

t7_vbox = widgets.VBox([t7title_label, t7r1_hbox, t7r2_hbox, t7r3_vbox], layout= maincontent_layout)

#### t8

t8title_label = widgets.Label(value= 'Export Products To Woocommerce CSV', style= bodytitlelabel_style, layout= bodytitlelabel_layout)
exportproductscsv_button = widgets.Button(description= 'Export CSV File', layout= bodybutton_layout)
exportproductscsvstatus_label = widgets.Label(value= '', style= bodystatuslabel_style)
t8_vbox = widgets.VBox([t8title_label, exportproductscsv_button, exportproductscsvstatus_label], layout= maincontent_layout)

####

tabs = [
    widgets.Box([t1_vbox],layout= mainbox_layout),
    widgets.Box([t2_hbox],layout= mainbox_layout),
    widgets.Box([t3_vbox],layout= mainbox_layout),
    widgets.Box([t4_vbox],layout= mainbox_layout),
    widgets.Box([t5_vbox],layout= mainbox_layout),
    widgets.Box([t6_vbox],layout= mainbox_layout),
    widgets.Box([t7_vbox],layout= mainbox_layout),
    widgets.Box([t8_vbox],layout= mainbox_layout)
]
tabs[0].layout.display = 'flex'

###

mainbox_r2v2_vbox = widgets.VBox(
    tabs,
    layout= collayout34
)
state.showntab = 0

###
row2_hbox = widgets.HBox(
    [
        r2v1_vbox, 
        mainbox_r2v2_vbox
    ],
    layout= row2_layout
)

#### event listners
##### menu

def beforetab3(): loadmappingconfigstatus_label.value = ''
def beforetab5(): loadproductsxmlstatus_label.value = ''
def befortab8(): exportproductscsvstatus_label.value = ''

tabcallbacks = [None, None, beforetab3, None, beforetab5, None, None, befortab8]
def beforetabchange(index):
    callback = tabcallbacks[index]
    if callback:
        callback()

def changetab(index):
    if state.showntab != index:
        beforetabchange(index)
        tabs[state.showntab].layout.display = 'none'
        tabs[index].layout.display = 'flex'
        state.showntab = index

configmenu_button.on_click(lambda b: changetab(0))
productmappingmenu_button.on_click(lambda b: changetab(1))
loadconfigmenu_button.on_click(lambda b: changetab(2))
savemappingconfigmenu_button.on_click(lambda b: changetab(3))

loadxmlproductsmenu_button.on_click(lambda b: changetab(4))
showproductsmenu_button.on_click(lambda b: changetab(5))
editproductspricesmenu_button.on_click(lambda b: changetab(6))
exportproductsmenu_button.on_click(lambda b: changetab(7))

##### top row events

def toprowsave_handler(b):
    if state.showntab == state.configtab:
        setappconfigfrominput()
    elif state.showntab == state.mappingtab:
        setwcmappingfrominput()
    state.configandmappingloadedfile = ""
    configandmappingset_handler()        
    r1v2save_button.disabled = True

def configandmappingset_handler():
    r1v2configuration_label.value = state.configandmappingloadedfile if state.configandmappingloadedfile else 'Configuration and mapping set manually.'
    if state.productxmlfilename:
        configandproductxmlset_handler()

def productsxmlfileloaded_handler():
    r1v2xmlfile_label.value = state.productxmlfilename
    if state.mappingconfigset:
        configandproductxmlset_handler()

def configandproductxmlset_handler():
    if state.mappingconfigset and state.productxmlfilename:
        makeproductsview()
    showproductsmenu_button.disabled = False
    editproductspricesmenu_button.disabled = False
    exportproductsmenu_button.disabled = False

r1v2save_button.on_click(toprowsave_handler)

##### t1 and t2 events

def configmappingchange_handler(change):
    if r1v2save_button.disabled and (state.showntab == state.configtab or state.showntab == state.mappingtab):
        r1v2save_button.disabled = False

mapsimple_checkbox.observe(configmappingchange_handler, 'value')
mapvariable_checkbox.observe(configmappingchange_handler, 'value')
variationmapping_dropdown.observe(configmappingchange_handler, 'value')
categoryseparator_text.observe(configmappingchange_handler, 'value')
attributemethod_dropdown.observe(configmappingchange_handler, 'value')
productnode_t21_text.observe(configmappingchange_handler, 'value')
type_t22_text.observe(configmappingchange_handler, 'value')
id_t23_text.observe(configmappingchange_handler, 'value')
sku_t24_text.observe(configmappingchange_handler, 'value')
name_t25_text.observe(configmappingchange_handler, 'value')
featured_t26_text.observe(configmappingchange_handler, 'value')
visible_t27_text.observe(configmappingchange_handler, 'value')
shortdesc_t28_text.observe(configmappingchange_handler, 'value')
description_t29_text.observe(configmappingchange_handler, 'value')
stock_t210_text.observe(configmappingchange_handler, 'value')
weight_t211_text.observe(configmappingchange_handler, 'value')
length_t212_text.observe(configmappingchange_handler, 'value')
width_t213_text.observe(configmappingchange_handler, 'value')
height_t214_text.observe(configmappingchange_handler, 'value')
note_t215_text.observe(configmappingchange_handler, 'value')
saleprice_t221_text.observe(configmappingchange_handler, 'value')
regularprice_t222_text.observe(configmappingchange_handler, 'value')
category_t223_text.observe(configmappingchange_handler, 'value')
tags_t2214_text.observe(configmappingchange_handler, 'value')
shippingclass_t224_text.observe(configmappingchange_handler, 'value')
featuredimage_t225_text.observe(configmappingchange_handler, 'value')
images_t226_text.observe(configmappingchange_handler, 'value')
featured_t26_text.observe(configmappingchange_handler, 'value')
option_t227_text.observe(configmappingchange_handler, 'value')
parentsku_t228_text.observe(configmappingchange_handler, 'value')
attibutelist_t229_text.observe(configmappingchange_handler, 'value')
attribute_t2210_text.observe(configmappingchange_handler, 'value')
attributename_t2211_text.observe(configmappingchange_handler, 'value')
attributevalue_t2212_text.observe(configmappingchange_handler, 'value')
attributevisible_t2213_text.observe(configmappingchange_handler, 'value')

##### t3 events

def validatemappingconfigfile(loadedconfig):
    try:
        if loadedconfig and loadedconfig['Mapping'] and loadedconfig['MappingOptions']:
            return True
    except:
        return False
    return False

def mappingconfigfilehandler(change):
    filename = next(iter(loadmappingconfig_fileupload.value))
    ecncodedjson = loadmappingconfig_fileupload.data[0]
    configjson = ecncodedjson.decode('utf-8')
    loadedconfig = json.loads(configjson)
    validation = validatemappingconfigfile(loadedconfig)
    if validation == True:
        setuploadedmappingconfig(loadedconfig)
        loadmappingconfigstatus_label.value = 'File uploaded successfully.'
        state.configandmappingloadedfile = filename
        state.mappingconfigset = True
        configandmappingset_handler()
    else:
        loadmappingconfigstatus_label.value = validation

    loadmappingconfig_fileupload.value.clear()  
    loadmappingconfig_fileupload._counter = 0

loadmappingconfig_fileupload.observe(mappingconfigfilehandler, names= 'value')

##### t4 events

def savemappingconfig_handler(b):
    os.makedirs('/content/proddo/config', exist_ok= True)
    savedfile = wcexportmappingconfigfile(state.mappingconfig, f'/content/proddo/config/proddo-mappingconfig-{nowstr()}.json')
    if savedfile:
        if env_colab:
            colab.files.download(savedfile)
    else:
        print('An error happend while saving config and mapping file.')

savemappingconfig_button.on_click(savemappingconfig_handler)

##### t5 events

def validateproductxmlfile(content):
    return True

def loadproductsxml_handler(change):
    filename = next(iter(loadproductsxml_fileupload.value))
    encodedxml = loadproductsxml_fileupload.data[0]
    xmldata = encodedxml.decode('utf-8')
    validation = validateproductxmlfile(xmldata)
    if validation == True:
        loadproductsxmlstatus_label.value = 'File uploaded successfully.'
        state.productxmlcontent = xmldata
        state.productxmlfilename = filename
        productsxmlfileloaded_handler()
    else:
        loadproductsxmlstatus_label.value = validation

    loadproductsxml_fileupload.value.clear()  
    loadproductsxml_fileupload._counter = 0

loadproductsxml_fileupload.observe(loadproductsxml_handler, names= 'value')

##### t6 events

def getproductlistforview(items):
    products = {"SKU": [], "Parent": [], "Name": [], "Regular Price": [], "Sale Price": []}
    for item in items:
        products["SKU"].append(item.find(state.mapping['SKU']).text)
        if state.mapping['ParentSKU']:
            parentsku = item.find(state.mapping['ParentSKU'])
            products["Parent"].append(item.find(state.mapping['ParentSKU']).text if parentsku else "")
        else: products["Parent"].append("")
        products["Name"].append(item.find(state.mapping['Name']).text)
        products["Regular Price"].append(item.find(state.mapping['RegularPrice']).text)
        if state.mapping['SalePrice']:
            saleprice = item.find(state.mapping['SalePrice'])
            products["Sale Price"].append(saleprice.text if saleprice else "")
        else: products["Sale Price"].append("")
    return products

def makeproductsview():
    if not logmaker(): return
    if state.mapping['ProductNode']:
        productsdataframe = xmlcall(state.productxmlcontent, state.mapping['ProductNode'], callback= getproductlistforview)
        dataframe = pd.DataFrame(productsdataframe)
        producttable = panel.widgets.Tabulator(dataframe, sizing_mode='stretch_width', disabled= True, pagination='local', page_size= 18)
        state.productstable = producttable
        with producttableoutput:
            producttableoutput.clear_output()
            display(producttable)

def updateproductpricesdf(items):
    prices = {"Regular Price": [], "Sale Price": []}
    i = 0
    for item in items:
        prices["Regular Price"].append((i, item.find(state.mapping['RegularPrice']).text))
        saleprice = None
        if state.mapping['SalePrice']:
            saleprice = item.find(state.mapping['SalePrice']) 
        saleprice = saleprice.text if saleprice else ""
        prices["Sale Price"].append((i, saleprice))
        i += 1
    return prices

def updateproductsview():
    if state.mapping['ProductNode']:
        prices = xmlcall(state.productxmlcontent, state.mapping['ProductNode'], callback= updateproductpricesdf)
        state.productstable.patch(prices)

##### t7 events

def editproductsprices_handler(b):
    pricenode = state.mapping['RegularPrice']
    salepricenode = state.mapping['SalePrice']
    pricechange = pricechange_text.value
    currencyrate = currencyrate_text.value
    reloadedcontent = wceditprices(content= state.productxmlcontent, pricenode= pricenode, salepricenode= salepricenode, pricechange= pricechange, currrate= currencyrate)
    state.productxmlcontent = reloadedcontent
    exporteditedproducts_button.disabled = False
    updateproductsview()
editproductsprices_button.on_click(editproductsprices_handler)

def exporteditedproducts_handler(b):
    filepath = f"/content/proddo/xml/{exlessbase(state.productxmlfilename)}-proddo-edited.xml"
    writetofile(state.productxmlcontent, filepath)
    if env_colab:
        colab.files.download(filepath)
    exporteditedproducts_button.disabled = True
exporteditedproducts_button.on_click(exporteditedproducts_handler)

##### t8 events

def exportproductscsv_handler(b):
    if not logmaker(): return
    if state.productxmlcontent:
        os.makedirs('/content/proddo/exports', exist_ok= True)
        csvoutput = f"/content/proddo/exports/proddo-export-{nowstr(True)}.csv"
        exportedfile = wcmapxmlstring(state.productxmlcontent, csvoutput, state.mappingconfig, defaultvalues)
        if exportedfile and env_colab:
            colab.files.download(exportedfile)
            exportproductscsvstatus_label.value = "The products where exported successfully."
    else:
        exportproductscsvstatus_label.value = "An error happend. The products weren't exported."

exportproductscsv_button.on_click(exportproductscsv_handler)

#### container
if logmaker():
    display(row1_hbox, row2_hbox)